In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from torch.utils.data import DataLoader
from models.dataset import LCDataset

In [2]:
data = pd.read_pickle('data/backup_data.pkl')
data.head()

,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,loan_status_num
Loan ID,,,,,,,,,,,,,,,,,,
273581de-85d8-4332-81a5-19b04ce68666,90a75dde-34d5-419c-90dc-1e58b04b3e35,Fully Paid,217646.0,Short Term,730.0,1184194.0,Home Mortgage,Debt Consolidation,10855.08,19.6,10.0,13.0,1.0,122170.0,272052.0,1.0,0.0,0
8af915d9-9e91-44a0-b5a2-564a45c12089,af534dea-d27e-4fd6-9de8-efaa52a78ec0,Fully Paid,548746.0,Short Term,678.0,2559110.0,Rent,Debt Consolidation,18660.28,22.6,33.0,4.0,0.0,437171.0,555038.0,0.0,0.0,0
2e841c8f-3dc1-464d-91c1-3d3d51e64c38,2ac05980-7848-4692-89ae-9321afe650f8,Fully Paid,234124.0,Short Term,727.0,693234.0,Rent,Debt Consolidation,14211.24,24.7,46.0,10.0,1.0,28291.0,107052.0,1.0,0.0,0
24e8c8bd-d10b-4dac-8b81-1da470ff5ecb,967e8733-7189-49b7-a3ab-6a1d0e1abdac,Fully Paid,666204.0,Long Term,723.0,1821967.0,Home Mortgage,Debt Consolidation,17612.24,22.0,34.0,15.0,0.0,813694.0,2004618.0,0.0,0.0,0
150ebbad-ebed-441e-b70d-2f350ad7dca6,40f729c9-54c7-4768-9fb5-2fa41d074c48,Charged Off,317108.0,Long Term,687.0,1133274.0,Rent,Debt Consolidation,9632.81,17.4,53.0,4.0,0.0,60287.0,126940.0,0.0,0.0,1


In [317]:
data2 = pd.read_pickle('data/backup_data.pkl')
data2.columns

Index(['Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Home Ownership', 'Purpose',
       'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens', 'loan_status_num'],
      dtype='object')

In [3]:
data['loan_status_num'].sum()
66242-16214

50028

In [4]:
data.drop(['Loan Status', 'Customer ID'], axis=1, inplace=True)

In [5]:
print([column for column in data.columns if data[column].dtype == object])

['Term', 'Home Ownership', 'Purpose']


In [6]:
dummies = ['Term', 'Home Ownership', 'Purpose']
data = pd.get_dummies(data, columns=dummies, drop_first=True)

In [7]:
data.head()

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
Loan ID,,,,,,,,,,,,,,,,,,,,,
273581de-85d8-4332-81a5-19b04ce68666,217646.0,730.0,1184194.0,10855.08,19.6,10.0,13.0,1.0,122170.0,272052.0,...,0,0,0,0,0,0,0,0,0,0
8af915d9-9e91-44a0-b5a2-564a45c12089,548746.0,678.0,2559110.0,18660.28,22.6,33.0,4.0,0.0,437171.0,555038.0,...,0,0,0,0,0,0,0,0,0,0
2e841c8f-3dc1-464d-91c1-3d3d51e64c38,234124.0,727.0,693234.0,14211.24,24.7,46.0,10.0,1.0,28291.0,107052.0,...,0,0,0,0,0,0,0,0,0,0
24e8c8bd-d10b-4dac-8b81-1da470ff5ecb,666204.0,723.0,1821967.0,17612.24,22.0,34.0,15.0,0.0,813694.0,2004618.0,...,0,0,0,0,0,0,0,0,0,0
150ebbad-ebed-441e-b70d-2f350ad7dca6,317108.0,687.0,1133274.0,9632.81,17.4,53.0,4.0,0.0,60287.0,126940.0,...,0,0,0,0,0,0,0,0,0,0


# Preparing data for the models

In [8]:
X = data.drop(columns=['loan_status_num'])
y = data['loan_status_num']
X_alt = data.drop(columns=['loan_status_num'])
X_alt_train, X_alt_temp, y_alt_train, y_alt_temp = train_test_split(X_alt, y, test_size=0.3, random_state=42)
X_train_extra, X_test_extra, y_train_extra, y_test_extra = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()

In [9]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)

X_validation, X_temp2, y_validation, y_temp2 = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

X_calibration, X_test, y_calibration, y_test = train_test_split(X_temp2, y_temp2, test_size=0.5, random_state=42)

X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)
X_calibration_scaled = scaler.transform(X_calibration)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_validation_tensor = torch.tensor(X_validation_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
X_calibration_tensor = torch.tensor(X_calibration_scaled, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(len(y_train),1)
y_validation_tensor = torch.tensor(y_validation.values, dtype=torch.float32).reshape(len(y_validation),1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(len(y_test),1)
y_calibration_tensor = torch.tensor(y_calibration.values, dtype=torch.float32).reshape(len(y_calibration),1)

In [10]:
from sklearn.tree import DecisionTreeClassifier, export_text
dt_model = DecisionTreeClassifier(max_depth=2, class_weight='balanced')
dt_model.fit(X_train_extra, y_train_extra)
tree_rules = export_text(dt_model, feature_names=list(X_train_extra.columns))
print(tree_rules)
predictions_extra = dt_model.predict(X_test_extra)

class_report = classification_report(y_test_extra, predictions_extra)
print("Classification Report:\n", class_report)


|--- Credit Score <= 684.50
|   |--- Annual Income <= 2489636.50
|   |   |--- class: 1
|   |--- Annual Income >  2489636.50
|   |   |--- class: 0
|--- Credit Score >  684.50
|   |--- Annual Income <= 1271242.50
|   |   |--- class: 1
|   |--- Annual Income >  1271242.50
|   |   |--- class: 0

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.44      0.57      7509
           1       0.27      0.67      0.39      2363

    accuracy                           0.49      9872
   macro avg       0.54      0.56      0.48      9872
weighted avg       0.68      0.49      0.53      9872


In [11]:
classlabels = torch.tensor(data['loan_status_num'].values, dtype=torch.long)
class_counts = torch.bincount(classlabels)

class_weights_original = 1.0 / class_counts.float()

class_weights_original /= class_weights_original.sum()
class_weights = class_weights_original[y_train_tensor.long()]


In [12]:
def evaluate_nn(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

def plot_learning_evolution(r):
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(r.history['loss'], label='Loss')
    plt.plot(r.history['val_loss'], label='val_Loss')
    plt.title('Loss evolution during trainig')
    plt.legend()

    plt.subplot(2, 2, 2)
    plt.plot(r.history['AUC'], label='AUC')
    plt.plot(r.history['val_AUC'], label='val_AUC')
    plt.title('AUC score evolution during trainig')
    plt.legend();


# Logistic Regression

In [13]:
from models.model import Logistic_Regression, MLP
from torchinfo import summary

In [14]:
n_features=len(data.drop(columns=['loan_status_num']).columns)
n_features

31

In [15]:
lr_model = Logistic_Regression(num_features=n_features)
summary(lr_model, input_size=X_train.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Logistic_Regression                      [23034, 1]                --
├─Sequential: 1-1                        [23034, 1]                --
│    └─Linear: 2-1                       [23034, 1]                32
│    └─Sigmoid: 2-2                      [23034, 1]                --
Total params: 32
Trainable params: 32
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.74
Input size (MB): 2.86
Forward/backward pass size (MB): 0.18
Params size (MB): 0.00
Estimated Total Size (MB): 3.04

In [16]:
LEARNING_RATE = 0.01
EPOCHS = 1000

In [17]:
loss_function = nn.BCELoss(weight=class_weights)

In [18]:
optimizer = optim.Adam(lr_model.parameters(), lr=LEARNING_RATE)

In [19]:
def calculate_accuracy(preds, actuals):

    with torch.no_grad():
        rounded_preds = torch.round(preds)
        num_correct = torch.sum(rounded_preds == actuals)
        accuracy = num_correct/len(preds)

    return accuracy

In [20]:
train_losses = []
#test_losses  = []
train_accs = []
test_accs  = []

for epoch in range(EPOCHS):
    train_preds = lr_model(X_train_tensor)
    train_loss  = loss_function(train_preds, y_train_tensor)

    with torch.no_grad():
        test_preds = lr_model(X_validation_tensor)
        #test_loss  = loss_function(test_preds, y_test)

    train_acc = calculate_accuracy(train_preds, y_train_tensor)
    test_acc  = calculate_accuracy(test_preds, y_validation_tensor)

    optimizer.zero_grad()
    train_loss.backward()

    optimizer.step()

    train_losses.append(train_loss.item())
    #test_losses.append(test_loss.item())
    train_accs.append(train_acc.item())
    test_accs.append(test_acc.item())

    if epoch%100==0:
        print(f'Epoch: {epoch} \t|' \
              f' Train loss: {np.round(train_loss.item(),3)} \t|' \
                  #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
              f' Train acc: {np.round(train_acc.item(),2)} \t|' \
              f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.263 	| Train acc: 0.53 	| Test acc: 0.53
Epoch: 100 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 200 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 300 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 400 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 500 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 600 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 700 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 800 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59
Epoch: 900 	| Train loss: 0.246 	| Train acc: 0.58 	| Test acc: 0.59


In [21]:
evaluate_nn(y_train_tensor.clone().detach(), train_preds.clone().detach().round(), train=True)
evaluate_nn(y_validation_tensor.clone().detach(), test_preds.clone().detach().round(), train=False)

Train Result:
Accuracy Score: 58.06%
_______________________________________________
CLASSIFICATION REPORT:
                    0.0          1.0  accuracy     macro avg  weighted avg
precision      0.815339     0.304955  0.580577      0.560147      0.692850
recall         0.579344     0.584479  0.580577      0.581912      0.580577
f1-score       0.677375     0.400794  0.580577      0.539085      0.610998
support    17506.000000  5528.000000  0.580577  23034.000000  23034.000000
_______________________________________________
Confusion Matrix: 
 [[10142  7364]
 [ 2297  3231]]

Test Result:
Accuracy Score: 59.38%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.828379    0.310041  0.593798     0.569210      0.706951
recall        0.592221    0.598952  0.593798     0.595586      0.593798
f1-score      0.690671    0.408583  0.593798     0.549627      0.624588
support    2494.000000

In [22]:
test_preds_lr = lr_model(X_test_tensor)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_lr.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 57.53%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.801913    0.312214  0.575272     0.557064      0.679044
recall        0.575061    0.575904  0.575272     0.575482      0.575272
f1-score      0.669800    0.404913  0.575272     0.537357      0.603338
support    2478.000000  830.000000  0.575272  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1425 1053]
 [ 352  478]]


# Semantic Loss in Logistic Regression

In [23]:
from importlib import reload
import models.loss
reload(models.loss)
from models.loss import semantic_loss

In [164]:
sl_model = Logistic_Regression(num_features=n_features)
summary(sl_model, input_size=X_train.shape)

Layer (type:depth-idx)                   Output Shape              Param #
Logistic_Regression                      [23034, 1]                --
├─Sequential: 1-1                        [23034, 1]                --
│    └─Linear: 2-1                       [23034, 1]                32
│    └─Sigmoid: 2-2                      [23034, 1]                --
Total params: 32
Trainable params: 32
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.74
Input size (MB): 2.86
Forward/backward pass size (MB): 0.18
Params size (MB): 0.00
Estimated Total Size (MB): 3.04

In [165]:
optimizer_sl = optim.Adam(sl_model.parameters(), lr=LEARNING_RATE)

In [166]:
credit_score = torch.tensor(X_alt_train['Credit Score'].values, dtype=torch.float32).reshape(len(y_train),1)
short_term = torch.tensor(X_alt_train['Term_Short Term'].values, dtype=torch.float32).reshape(len(y_train),1)
annual_income = torch.tensor(X_alt_train['Annual Income'].values, dtype=torch.float32).reshape(len(y_train),1)


rule = torch.logical_or(torch.logical_and(credit_score<=684.5,  annual_income<=2489636.5), torch.logical_and(credit_score>684.5, annual_income<=1271242.5))
rule=rule.float()

In [167]:
print(rule)

tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [0.],
        [0.]])


In [168]:
train_losses_sl = []
#test_losses  = []
train_accs_sl = []
test_accs_sl  = []

for epoch in range(EPOCHS):
    train_preds = sl_model(X_train_tensor)
    #train_loss  = loss_function(train_preds, y_train_tensor)
    train_loss = semantic_loss(train_preds, y_train_tensor, rule, class_weights, 0.01)

    with torch.no_grad():
        test_preds = sl_model(X_validation_tensor)
        #test_loss  = loss_function(test_preds, y_test)

    train_acc = calculate_accuracy(train_preds, y_train_tensor)
    test_acc  = calculate_accuracy(test_preds, y_validation_tensor)

    optimizer_sl.zero_grad()
    train_loss.backward()

    optimizer_sl.step()

    train_losses_sl.append(train_loss.item())
    #test_losses.append(test_loss.item())
    train_accs_sl.append(train_acc.item())
    test_accs_sl.append(test_acc.item())

    if epoch%100==0:
        print(f'Epoch: {epoch} \t|' \
              f' Train loss: {np.round(train_loss.item(),3)} \t|' \
                  #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
              f' Train acc: {np.round(train_acc.item(),2)} \t|' \
              f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.185 	| Train acc: 0.55 	| Test acc: 0.56
Epoch: 100 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.56
Epoch: 200 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 300 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 400 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 500 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 600 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 700 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 800 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 900 	| Train loss: 0.157 	| Train acc: 0.55 	| Test acc: 0.55


In [169]:
evaluate_nn(y_train_tensor.clone().detach(), train_preds.clone().detach().round(), train=True)
evaluate_nn(y_validation_tensor.clone().detach(), test_preds.clone().detach().round(), train=False)

Train Result:
Accuracy Score: 54.59%
_______________________________________________
CLASSIFICATION REPORT:
                    0.0          1.0  accuracy     macro avg  weighted avg
precision      0.820389     0.295149  0.545889      0.557769      0.694335
recall         0.515309     0.642728  0.545889      0.579018      0.545889
f1-score       0.633008     0.404531  0.545889      0.518770      0.578175
support    17506.000000  5528.000000  0.545889  23034.000000  23034.000000
_______________________________________________
Confusion Matrix: 
 [[9021 8485]
 [1975 3553]]

Test Result:
Accuracy Score: 55.45%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.833227    0.296690  0.554498     0.564959      0.707535
recall        0.522855    0.657929  0.554498     0.590392      0.554498
f1-score      0.642523    0.408961  0.554498     0.525742      0.587808
support    2494.000000  76

In [170]:
test_preds_sl = sl_model(X_test_tensor)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_sl.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 54.75%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.812222    0.308002  0.547461     0.560112      0.685709
recall        0.514931    0.644578  0.547461     0.579755      0.547461
f1-score      0.630279    0.416829  0.547461     0.523554      0.576723
support    2478.000000  830.000000  0.547461  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1276 1202]
 [ 295  535]]


In [57]:
credit_score_test = torch.tensor(X_test['Credit Score'].values, dtype=torch.float32).reshape(len(X_test), 1)
short_term_test = torch.tensor(X_test['Term_Short Term'].values, dtype=torch.float32).reshape(len(X_test), 1)
annual_income_test = torch.tensor(X_test['Annual Income'].values, dtype=torch.float32).reshape(len(X_test), 1)

rule_test = torch.logical_or(
    torch.logical_and(credit_score_test <= 684.5, annual_income_test <= 2489636.5),
    torch.logical_and(credit_score_test > 684.5, annual_income_test <= 1271242.5)
)
rule_matched = torch.nonzero(rule_test.float(), as_tuple=True)[0].tolist()
yespred = torch.nonzero(test_preds_sl.round(), as_tuple=True)[0].tolist()
same = [value for value in yespred if value in rule_matched]
len(same)

1918

In [58]:
test_preds_sl.shape

torch.Size([3308, 1])

In [59]:
rounded_preds_lr = torch.round(test_preds_lr)
rounded_preds_sl = torch.round(test_preds_sl)

different_predictions = (rounded_preds_lr != rounded_preds_sl)

correct_predictions_lr = (rounded_preds_lr == y_test_tensor)
correct_predictions_sl = (rounded_preds_sl == y_test_tensor)

different_and_correct_lr = different_predictions & correct_predictions_lr
different_and_correct_sl = different_predictions & correct_predictions_sl

#indices where predictions are different and correct
indices_different_and_correct_lr = torch.nonzero(different_and_correct_lr).flatten()
indices_different_and_correct_sl = torch.nonzero(different_and_correct_sl).flatten()

In [60]:
only_sl_correct = torch.nonzero(different_and_correct_sl.float(), as_tuple=True)[0].tolist()
only_lr_correct = torch.nonzero(different_and_correct_lr.float(), as_tuple=True)[0].tolist()

In [61]:
X_test.iloc[only_sl_correct]
#y_test.iloc[only_sl_correct]


,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
Loan ID,,,,,,,,,,,,,,,,,,,,,
8deb9448-0939-4b4a-add5-89f4442fddf7,155628.0,645.0,1263956.0,10216.87,13.5,43.0,12.0,0.0,174781.0,417274.0,...,0,0,0,0,0,0,0,0,0,0
b4f3602e-8187-4076-966a-cfd71084e8f4,161810.0,724.0,1397507.0,21311.92,18.0,5.0,12.0,0.0,108395.0,189860.0,...,0,0,0,0,0,0,0,0,0,0
1103946b-1671-4c39-9bf5-923c10ce0cb1,130064.0,726.0,1310582.0,18784.92,14.0,25.0,15.0,0.0,536769.0,1195260.0,...,0,0,0,0,0,0,0,0,0,0
8040dec3-ef55-4baa-bc20-1c2853ccace7,117282.0,734.0,489953.0,7961.76,13.8,9.0,13.0,0.0,195225.0,314402.0,...,0,0,0,0,0,0,0,0,0,0
f4ce26ef-0e52-475c-9d62-75a78c68a322,215578.0,723.0,1008444.0,18236.01,22.5,42.0,13.0,0.0,569943.0,920414.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9baefe4e-d64e-4db9-a303-5ad61a7b7381,220528.0,664.0,914185.0,13103.35,32.5,6.0,8.0,0.0,78261.0,187594.0,...,0,1,0,0,0,0,0,0,0,0
1471a3f6-63ed-43b4-9b21-58d3ce05b0a1,251944.0,744.0,946010.0,12613.53,11.9,13.0,22.0,0.0,170050.0,674344.0,...,0,0,0,0,0,0,0,0,0,0
33299529-29f2-4eb1-a730-16ac4ef0d57e,133672.0,707.0,423263.0,9488.22,45.5,26.0,9.0,0.0,223820.0,362956.0,...,0,0,0,0,0,1,0,0,0,0


In [62]:
X_train[X_train['Credit Score']>850]

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
Loan ID,,,,,,,,,,,,,,,,,,,,,


In [63]:
rule_matched = torch.nonzero(rule, as_tuple=True)[0].tolist()

In [64]:
intersection_sl = [value for value in only_sl_correct if value in rule_matched]
intersection_lr = [value for value in only_lr_correct if value in rule_matched]

In [65]:
X_test.iloc[intersection_sl]

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,...,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Purpose_wedding
Loan ID,,,,,,,,,,,,,,,,,,,,,
8deb9448-0939-4b4a-add5-89f4442fddf7,155628.0,645.0,1263956.0,10216.87,13.5,43.0,12.0,0.0,174781.0,417274.0,...,0,0,0,0,0,0,0,0,0,0
f4ce26ef-0e52-475c-9d62-75a78c68a322,215578.0,723.0,1008444.0,18236.01,22.5,42.0,13.0,0.0,569943.0,920414.0,...,0,0,0,0,0,0,0,0,0,0
1f5af938-7a12-4a47-a7b4-6ffdce0f25a6,462704.0,735.0,2473705.0,26179.91,27.9,43.0,13.0,0.0,622136.0,1549174.0,...,0,0,0,0,0,0,0,0,0,0
36909248-ba22-4b16-8e2f-a238229064f8,51744.0,706.0,1042606.0,3371.17,41.8,35.0,8.0,0.0,29545.0,53878.0,...,0,0,0,0,0,0,0,0,0,0
a35bc423-f2d0-4300-860c-39695faccf8f,388520.0,639.0,3914589.0,51215.83,16.0,10.0,24.0,0.0,292448.0,314710.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5cf1e7c7-8b29-4495-a778-540275568d1d,108130.0,737.0,644290.0,9020.06,13.0,6.0,11.0,0.0,108547.0,287606.0,...,0,0,0,0,0,0,0,0,0,0
d557a41b-14fb-4a3f-b4eb-1d35d7325dea,224818.0,745.0,1359127.0,22312.46,15.4,63.0,9.0,1.0,108262.0,220330.0,...,0,0,0,0,0,0,0,0,0,0
19bac196-ba5a-4473-980a-1d351c02f98f,659934.0,714.0,2754867.0,39716.08,29.2,24.0,8.0,0.0,926478.0,1347698.0,...,0,0,0,0,0,0,0,0,0,0


# Multilayer perceptrons

In [66]:
from importlib import reload  
import models.loss
reload(models.loss)
from models.loss import semantic_loss

In [67]:
from importlib import reload  
import models.model
reload(models.model)
from models.model import MLP

In [68]:
credit_score_index = X_alt_train.columns.get_loc('Credit Score')
short_term_index = X_alt_train.columns.get_loc('Term_Short Term')
annual_income_index = X_alt_train.columns.get_loc('Annual Income')

In [69]:
dataset = LCDataset(X_train_tensor, y_train_tensor)

data_loader = DataLoader(dataset, batch_size=5000, shuffle=True)

In [70]:
mlp_model = MLP(num_features=n_features)
summary(mlp_model, input_size=X_train.shape)

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [23034, 1]                --
├─Sequential: 1-1                        [23034, 1]                --
│    └─Linear: 2-1                       [23034, 3]                96
│    └─ReLU: 2-2                         [23034, 3]                --
│    └─Linear: 2-3                       [23034, 1]                4
│    └─Sigmoid: 2-4                      [23034, 1]                --
Total params: 100
Trainable params: 100
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.30
Input size (MB): 2.86
Forward/backward pass size (MB): 0.74
Params size (MB): 0.00
Estimated Total Size (MB): 3.59

In [71]:
optimizer_mlp = optim.Adam(mlp_model.parameters())

In [72]:
train_losses_mlp = []
#test_losses  = []
train_accs_mlp = []
test_accs_mlp  = []

for epoch in range(200):
    for features, labels in data_loader:
        train_preds_mlp = mlp_model(features)
        class_weights_batch = class_weights_original[labels.long()]
        loss_function = nn.BCELoss(weight=class_weights_batch)

        train_loss_mlp  = loss_function(train_preds_mlp, labels)

        with torch.no_grad():
            test_preds_mlp = mlp_model(X_validation_tensor)
            #test_loss  = loss_function(test_preds, y_test)

        train_acc = calculate_accuracy(train_preds_mlp, labels)
        test_acc  = calculate_accuracy(test_preds_mlp, y_validation_tensor)

        optimizer_mlp.zero_grad()
        train_loss_mlp.backward()

        optimizer_mlp.step()

        train_losses_mlp.append(train_loss_mlp.item())
        #test_losses.append(test_loss.item())
        train_accs_mlp.append(train_acc.item())
        test_accs_mlp.append(test_acc.item())

    #if epoch%100==0:
    print(f'Epoch: {epoch} \t|' \
          f' Train loss: {np.round(train_loss_mlp.item(),3)} \t|' \
              #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
          f' Train acc: {np.round(train_acc.item(),2)} \t|' \
          f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.256 	| Train acc: 0.3 	| Test acc: 0.29
Epoch: 1 	| Train loss: 0.26 	| Train acc: 0.31 	| Test acc: 0.3
Epoch: 2 	| Train loss: 0.256 	| Train acc: 0.32 	| Test acc: 0.31
Epoch: 3 	| Train loss: 0.256 	| Train acc: 0.34 	| Test acc: 0.32
Epoch: 4 	| Train loss: 0.257 	| Train acc: 0.35 	| Test acc: 0.34
Epoch: 5 	| Train loss: 0.255 	| Train acc: 0.36 	| Test acc: 0.35
Epoch: 6 	| Train loss: 0.251 	| Train acc: 0.37 	| Test acc: 0.37
Epoch: 7 	| Train loss: 0.25 	| Train acc: 0.38 	| Test acc: 0.38
Epoch: 8 	| Train loss: 0.253 	| Train acc: 0.4 	| Test acc: 0.4
Epoch: 9 	| Train loss: 0.256 	| Train acc: 0.42 	| Test acc: 0.42
Epoch: 10 	| Train loss: 0.246 	| Train acc: 0.43 	| Test acc: 0.44
Epoch: 11 	| Train loss: 0.255 	| Train acc: 0.46 	| Test acc: 0.46
Epoch: 12 	| Train loss: 0.253 	| Train acc: 0.47 	| Test acc: 0.48
Epoch: 13 	| Train loss: 0.254 	| Train acc: 0.48 	| Test acc: 0.49
Epoch: 14 	| Train loss: 0.255 	| Train acc: 0.52 	| Test acc: 0

In [73]:
evaluate_nn(y_validation_tensor.clone().detach(), test_preds_mlp.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 60.18%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.819541    0.307081  0.601781     0.563311      0.699490
recall        0.615477    0.557012  0.601781     0.586244      0.601781
f1-score      0.703000    0.395901  0.601781     0.549451      0.631057
support    2494.000000  763.000000  0.601781  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[1535  959]
 [ 338  425]]


In [74]:
test_preds_mlp = mlp_model(X_test_tensor)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_mlp.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 59.19%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.799363    0.317416  0.591898     0.558389      0.678439
recall        0.607748    0.544578  0.591898     0.576163      0.591898
f1-score      0.690509    0.401065  0.591898     0.545787      0.617885
support    2478.000000  830.000000  0.591898  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1506  972]
 [ 378  452]]


In [171]:
mlpsl_model = MLP(num_features=n_features)
summary(mlpsl_model, input_size=X_train.shape)

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [23034, 1]                --
├─Sequential: 1-1                        [23034, 1]                --
│    └─Linear: 2-1                       [23034, 3]                96
│    └─ReLU: 2-2                         [23034, 3]                --
│    └─Linear: 2-3                       [23034, 1]                4
│    └─Sigmoid: 2-4                      [23034, 1]                --
Total params: 100
Trainable params: 100
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 2.30
Input size (MB): 2.86
Forward/backward pass size (MB): 0.74
Params size (MB): 0.00
Estimated Total Size (MB): 3.59

In [172]:
optimizer_mlpsl = optim.Adam(mlpsl_model.parameters())

In [173]:
train_losses_mlpsl = []
#test_losses  = []
train_accs_mlpsl = []
test_accs_mlpsl  = []

for epoch in range(200):
    for features, labels in data_loader:
        train_preds_mlpsl = mlpsl_model(features)
        class_weights_batch = class_weights_original[labels.long()]
        loss_function = nn.BCELoss(weight=class_weights_batch)
        credit_score_mlpsl = features[:, credit_score_index].reshape(-1, 1)
        short_term_mlpsl = features[:, short_term_index].reshape(-1, 1)
        annual_income_mlpsl = features[:, annual_income_index].reshape(-1, 1)
        
        rule_mlpsl = torch.logical_or(credit_score_mlpsl>3300.5, torch.logical_and(credit_score_mlpsl<=3300.5,torch.logical_and(short_term_mlpsl<=0.5, annual_income_mlpsl<=1405107))).float()
        #train_loss_mlp  = loss_function(train_preds_mlp, labels)
        train_loss_mlpsl = semantic_loss(train_preds_mlpsl, labels, rule_mlpsl, class_weights_batch, 0.005)

        with torch.no_grad():
            test_preds_mlpsl = mlpsl_model(X_validation_tensor)
            #test_loss  = loss_function(test_preds, y_test)

        train_acc = calculate_accuracy(train_preds_mlpsl, labels)
        test_acc  = calculate_accuracy(test_preds_mlpsl, y_validation_tensor)

        optimizer_mlpsl.zero_grad()
        train_loss_mlpsl.backward()

        optimizer_mlpsl.step()

        train_losses_mlpsl.append(train_loss_mlpsl.item())
        #test_losses.append(test_loss.item())
        train_accs_mlpsl.append(train_acc.item())
        test_accs_mlpsl.append(test_acc.item())

    #if epoch%100==0:
    print(f'Epoch: {epoch} \t|' \
          f' Train loss: {np.round(train_loss_mlpsl.item(),3)} \t|' \
              #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
          f' Train acc: {np.round(train_acc.item(),2)} \t|' \
          f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.225 	| Train acc: 0.57 	| Test acc: 0.56
Epoch: 1 	| Train loss: 0.221 	| Train acc: 0.55 	| Test acc: 0.55
Epoch: 2 	| Train loss: 0.22 	| Train acc: 0.53 	| Test acc: 0.54
Epoch: 3 	| Train loss: 0.225 	| Train acc: 0.51 	| Test acc: 0.53
Epoch: 4 	| Train loss: 0.223 	| Train acc: 0.5 	| Test acc: 0.51
Epoch: 5 	| Train loss: 0.223 	| Train acc: 0.49 	| Test acc: 0.5
Epoch: 6 	| Train loss: 0.219 	| Train acc: 0.49 	| Test acc: 0.49
Epoch: 7 	| Train loss: 0.223 	| Train acc: 0.48 	| Test acc: 0.49
Epoch: 8 	| Train loss: 0.222 	| Train acc: 0.48 	| Test acc: 0.48
Epoch: 9 	| Train loss: 0.221 	| Train acc: 0.51 	| Test acc: 0.48
Epoch: 10 	| Train loss: 0.22 	| Train acc: 0.5 	| Test acc: 0.48
Epoch: 11 	| Train loss: 0.224 	| Train acc: 0.5 	| Test acc: 0.49
Epoch: 12 	| Train loss: 0.221 	| Train acc: 0.5 	| Test acc: 0.49
Epoch: 13 	| Train loss: 0.22 	| Train acc: 0.49 	| Test acc: 0.5
Epoch: 14 	| Train loss: 0.219 	| Train acc: 0.5 	| Test acc: 0.5
E

In [174]:
evaluate_nn(y_validation_tensor.clone().detach(), test_preds_mlpsl.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 58.89%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.832470    0.310526  0.588885     0.571498      0.710197
recall        0.579791    0.618611  0.588885     0.599201      0.588885
f1-score      0.683526    0.413491  0.588885     0.548509      0.620267
support    2494.000000  763.000000  0.588885  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[1446 1048]
 [ 291  472]]


In [175]:
test_preds_mlpsl = mlpsl_model(X_test_tensor)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_mlpsl.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 57.53%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.806047    0.315113  0.575272     0.560580      0.682868
recall        0.570218    0.590361  0.575272     0.580290      0.575272
f1-score      0.667927    0.410901  0.575272     0.539414      0.603438
support    2478.000000  830.000000  0.575272  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1413 1065]
 [ 340  490]]


In [104]:
credit_score_test = torch.tensor(X_test['Credit Score'].values, dtype=torch.float32).reshape(len(X_test), 1)
short_term_test = torch.tensor(X_test['Term_Short Term'].values, dtype=torch.float32).reshape(len(X_test), 1)
annual_income_test = torch.tensor(X_test['Annual Income'].values, dtype=torch.float32).reshape(len(X_test), 1)

rule_test = torch.logical_or(
    torch.logical_and(credit_score_test <= 684.5, annual_income_test <= 2489636.5),
    torch.logical_and(credit_score_test > 684.5, annual_income_test <= 1271242.5)
)
rule_matched = torch.nonzero(rule_test.float(), as_tuple=True)[0].tolist()
yespred = torch.nonzero(test_preds_mlpsl.round(), as_tuple=True)[0].tolist()
same = [value for value in yespred if value in rule_matched]
len(same)
len(rule_matched)

1952

# Tree-based predictions

In [105]:
dt_model = DecisionTreeClassifier(class_weight='balanced')
dt_model.fit(X_train_scaled, y_train)

predictions_dt = dt_model.predict(X_validation_scaled)

class_report = classification_report(y_validation, predictions_dt)
evaluate_nn(y_validation_tensor.clone().detach(), predictions_dt.round(), train=False)

Test Result:
Accuracy Score: 69.05%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.790689    0.325250  0.690513     0.557969      0.681653
recall        0.810345    0.298820  0.690513     0.554583      0.690513
f1-score      0.800396    0.311475  0.690513     0.555936      0.685859
support    2494.000000  763.000000  0.690513  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[2021  473]
 [ 535  228]]


In [106]:
test_preds_dt = dt_model.predict(X_test_scaled)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_dt.round(), train=False)

Test Result:
Accuracy Score: 69.74%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.781978    0.375907    0.6974     0.578942      0.680092
recall        0.826473    0.312048    0.6974     0.569261      0.697400
f1-score      0.803610    0.341014    0.6974     0.572312      0.687541
support    2478.000000  830.000000    0.6974  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[2048  430]
 [ 571  259]]


In [227]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train))

xgb_model.fit(X_train_scaled, y_train)

predictions_xgb = xgb_model.predict(X_validation_scaled)

class_report_xgb = classification_report(y_validation, predictions_xgb)
print("XGBoost Classification Report:\n", class_report_xgb)
evaluate_nn(y_validation_tensor.clone().detach(), predictions_xgb.round(), train=False)


XGBoost Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.71      0.75      2494
           1       0.31      0.44      0.37       763

    accuracy                           0.65      3257
   macro avg       0.56      0.57      0.56      3257
weighted avg       0.69      0.65      0.66      3257

Test Result:
Accuracy Score: 64.57%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.804269    0.314692  0.645686     0.559480      0.689578
recall        0.710104    0.435125  0.645686     0.572614      0.645686
f1-score      0.754259    0.365237  0.645686     0.559748      0.663125
support    2494.000000  763.000000  0.645686  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[1771  723]
 [ 431  332]]


In [228]:
test_preds_xgb = xgb_model.predict(X_test_scaled)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_xgb.round(), train=False)

Test Result:
Accuracy Score: 65.63%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.795765    0.352546  0.656288     0.574155      0.684558
recall        0.728006    0.442169  0.656288     0.585088      0.656288
f1-score      0.760379    0.392304  0.656288     0.576341      0.668027
support    2478.000000  830.000000  0.656288  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1804  674]
 [ 463  367]]


# Limited data

In [109]:
X_train.columns

Index(['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt',
       'Years of Credit History', 'Months since last delinquent',
       'Number of Open Accounts', 'Number of Credit Problems',
       'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies',
       'Tax Liens', 'Term_Short Term', 'Home Ownership_Home Mortgage',
       'Home Ownership_Own Home', 'Home Ownership_Rent', 'Purpose_Buy House',
       'Purpose_Buy a Car', 'Purpose_Debt Consolidation',
       'Purpose_Educational Expenses', 'Purpose_Home Improvements',
       'Purpose_Medical Bills', 'Purpose_Other', 'Purpose_Take a Trip',
       'Purpose_major_purchase', 'Purpose_moving', 'Purpose_other',
       'Purpose_renewable_energy', 'Purpose_small_business',
       'Purpose_vacation', 'Purpose_wedding'],
      dtype='object')

In [110]:
X_train_alt = X_train[['Current Loan Amount', 'Term_Short Term', 'Credit Score', 'Annual Income', 'Tax Liens', 'Years of Credit History']]
X_calib_alt = X_calibration[['Current Loan Amount', 'Term_Short Term', 'Credit Score', 'Annual Income', 'Tax Liens', 'Years of Credit History']]
X_train_lim, X_remain, y_train_lim, y_remain = train_test_split(X_train_alt, y_train, test_size=0.995, random_state=42)
X_train_lim_scaled = scaler.fit_transform(X_train_lim)
X_calibration_lim_scaled = scaler.transform(X_calib_alt)
X_validation_alt = X_validation[['Current Loan Amount', 'Term_Short Term', 'Credit Score', 'Annual Income', 'Tax Liens', 'Years of Credit History']]
X_validation_lim_scaled = scaler.transform(X_validation_alt)
X_validation_tensor_lim = torch.tensor(X_validation_lim_scaled, dtype=torch.float32)
X_calibration_tensor_lim = torch.tensor(X_calibration_lim_scaled, dtype=torch.float32)
X_train_tensor_lim = torch.tensor(X_train_lim_scaled, dtype=torch.float32)
y_train_tensor_lim = torch.tensor(y_train_lim.values, dtype=torch.float32).reshape(len(y_train_lim),1)
X_train_tensor_lim.shape
y_train_tensor_lim.shape

torch.Size([115, 1])

In [111]:
X_test_alt = X_test[['Current Loan Amount', 'Term_Short Term', 'Credit Score', 'Annual Income', 'Tax Liens', 'Years of Credit History']]
X_test_lim_scaled = scaler.transform(X_test_alt)
X_test_tensor_lim = torch.tensor(X_test_lim_scaled, dtype=torch.float32)

In [112]:
credit_score_index_lim = X_train_lim.columns.get_loc('Credit Score')
short_term_index_lim = X_train_lim.columns.get_loc('Term_Short Term')
annual_income_index_lim = X_train_lim.columns.get_loc('Annual Income')

In [113]:
credit_score_lim = torch.tensor(X_train_lim['Credit Score'].values, dtype=torch.float32).reshape(len(y_train_lim),1)
short_term_lim = torch.tensor(X_train_lim['Term_Short Term'].values, dtype=torch.float32).reshape(len(y_train_lim),1)
annual_income_lim = torch.tensor(X_train_lim['Annual Income'].values, dtype=torch.float32).reshape(len(y_train_lim),1)

#rule=credit_score>3300.5
rule_lim = torch.logical_or(torch.logical_and(credit_score_lim<=684.5,  annual_income_lim<=2489636.5), torch.logical_and(credit_score_lim>684.5, annual_income_lim<=1271242.5))
#rule_lim = torch.logical_or(credit_score_lim>3300.5, torch.logical_and(credit_score_lim<=3300.5,torch.logical_and(short_term_lim<=0.5, annual_income_lim<=1405107)))
rule_lim=rule_lim.float()

In [114]:
small_model_lr = Logistic_Regression(num_features=6)
optimizer_small_lr = optim.Adam(small_model_lr.parameters(), lr=LEARNING_RATE)
class_weights_lim = class_weights_original[y_train_tensor_lim.long()]
loss_function_lim=nn.BCELoss(weight=class_weights_lim)
train_losses = []
#test_losses  = []
train_accs = []
test_accs  = []

for epoch in range(EPOCHS):
    train_preds_lim = small_model_lr(X_train_tensor_lim)
    #train_loss  = loss_function(train_preds, y_train_tensor)

    #print(torch.min(train_preds_lim), torch.max(train_preds_lim))
    #train_loss = semantic_loss(train_preds_lim, y_train_tensor_lim, rule_lim, class_weights_lim, 0.03)
    train_loss = loss_function_lim(train_preds_lim, y_train_tensor_lim)

    with torch.no_grad():
        test_preds = small_model_lr(X_validation_tensor_lim)
        #test_loss  = loss_function(test_preds, y_test)

    train_acc = calculate_accuracy(train_preds_lim, y_train_tensor_lim)
    test_acc  = calculate_accuracy(test_preds, y_validation_tensor)

    optimizer_small_lr.zero_grad()
    train_loss.backward()

    optimizer_small_lr.step()

    train_losses.append(train_loss.item())
    #test_losses.append(test_loss.item())
    train_accs.append(train_acc.item())
    test_accs.append(test_acc.item())

    if epoch%100==0:
        print(f'Epoch: {epoch} \t|' \
              f' Train loss: {np.round(train_loss.item(),3)} \t|' \
                  #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
              f' Train acc: {np.round(train_acc.item(),2)} \t|' \
              f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.272 	| Train acc: 0.38 	| Test acc: 0.4
Epoch: 100 	| Train loss: 0.225 	| Train acc: 0.64 	| Test acc: 0.57
Epoch: 200 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 300 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 400 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 500 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 600 	| Train loss: 0.224 	| Train acc: 0.65 	| Test acc: 0.58
Epoch: 700 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 800 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58
Epoch: 900 	| Train loss: 0.224 	| Train acc: 0.64 	| Test acc: 0.58


In [115]:
test_preds_lim_lr = small_model_lr(X_test_tensor_lim)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_lim_lr.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 57.35%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.777431    0.290253  0.573458     0.533842      0.655195
recall        0.603309    0.484337  0.573458     0.543823      0.573458
f1-score      0.679391    0.362980  0.573458     0.521185      0.600001
support    2478.000000  830.000000  0.573458  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1495  983]
 [ 428  402]]


In [176]:
small_model = Logistic_Regression(num_features=6)
optimizer_small = optim.Adam(small_model.parameters(), lr=LEARNING_RATE)
class_weights_lim = class_weights_original[y_train_tensor_lim.long()]
loss_function_lim=nn.BCELoss(weight=class_weights_lim)
train_losses = []
#test_losses  = []
train_accs = []
test_accs  = []

for epoch in range(EPOCHS):
    train_preds_lim = small_model(X_train_tensor_lim)
    #train_loss  = loss_function(train_preds, y_train_tensor)

    #print(torch.min(train_preds_lim), torch.max(train_preds_lim))
    train_loss = semantic_loss(train_preds_lim, y_train_tensor_lim, rule_lim, class_weights_lim, 0.1)
    #train_loss = loss_function_lim(train_preds_lim, y_train_tensor_lim)

    with torch.no_grad():
        test_preds = small_model(X_validation_tensor_lim)
        #test_loss  = loss_function(test_preds, y_test)

    train_acc = calculate_accuracy(train_preds_lim, y_train_tensor_lim)
    test_acc  = calculate_accuracy(test_preds, y_validation_tensor)

    optimizer_small.zero_grad()
    train_loss.backward()

    optimizer_small.step()

    train_losses.append(train_loss.item())
    #test_losses.append(test_loss.item())
    train_accs.append(train_acc.item())
    test_accs.append(test_acc.item())

    if epoch%100==0:
        print(f'Epoch: {epoch} \t|' \
              f' Train loss: {np.round(train_loss.item(),3)} \t|' \
                  #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
              f' Train acc: {np.round(train_acc.item(),2)} \t|' \
              f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: -0.079 	| Train acc: 0.34 	| Test acc: 0.35
Epoch: 100 	| Train loss: -0.145 	| Train acc: 0.56 	| Test acc: 0.47
Epoch: 200 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 300 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 400 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 500 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 600 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 700 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 800 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47
Epoch: 900 	| Train loss: -0.145 	| Train acc: 0.55 	| Test acc: 0.47


In [177]:
evaluate_nn(y_train_tensor_lim.clone().detach(), train_preds_lim.clone().detach().round(), train=True)
evaluate_nn(y_validation_tensor.clone().detach(), test_preds.clone().detach().round(), train=False)

Train Result:
Accuracy Score: 54.78%
_______________________________________________
CLASSIFICATION REPORT:
                 0.0        1.0  accuracy   macro avg  weighted avg
precision   0.875000   0.313433  0.547826    0.594216      0.743154
recall      0.477273   0.777778  0.547826    0.627525      0.547826
f1-score    0.617647   0.446809  0.547826    0.532228      0.577537
support    88.000000  27.000000  0.547826  115.000000    115.000000
_______________________________________________
Confusion Matrix: 
 [[42 46]
 [ 6 21]]

Test Result:
Accuracy Score: 46.98%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.819317    0.265564  0.469757     0.542441      0.689592
recall        0.394547    0.715596  0.469757     0.555072      0.469757
f1-score      0.532612    0.387371  0.469757     0.459992      0.498587
support    2494.000000  763.000000  0.469757  3257.000000   3257.0000

In [178]:
test_preds_lim = small_model(X_test_tensor_lim)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_lim.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 46.92%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.789263    0.275243  0.469166     0.532253      0.660292
recall        0.397498    0.683133  0.469166     0.540315      0.469166
f1-score      0.528717    0.392388  0.469166     0.460552      0.494511
support    2478.000000  830.000000  0.469166  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[ 985 1493]
 [ 263  567]]


In [132]:
credit_score_test = torch.tensor(X_test['Credit Score'].values, dtype=torch.float32).reshape(len(X_test), 1)
short_term_test = torch.tensor(X_test['Term_Short Term'].values, dtype=torch.float32).reshape(len(X_test), 1)
annual_income_test = torch.tensor(X_test['Annual Income'].values, dtype=torch.float32).reshape(len(X_test), 1)

rule_test = torch.logical_or(
    torch.logical_and(credit_score_test <= 684.5, annual_income_test <= 2489636.5),
    torch.logical_and(credit_score_test > 684.5, annual_income_test <= 1271242.5)
)
rule_matched = torch.nonzero(rule_test.float(), as_tuple=True)[0].tolist()
yespred = torch.nonzero(test_preds_lim.round(), as_tuple=True)[0].tolist()
same = [value for value in yespred if value in rule_matched]
len(same)

1234

In [133]:
dataset_lim = LCDataset(X_train_tensor_lim, y_train_tensor_lim)

data_loader_lim = DataLoader(dataset_lim, batch_size=300, shuffle=True)

In [134]:
mlp_model_lim = MLP(num_features=6)
summary(mlp_model_lim, input_size=X_train_lim.shape)

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [115, 1]                  --
├─Sequential: 1-1                        [115, 1]                  --
│    └─Linear: 2-1                       [115, 3]                  21
│    └─ReLU: 2-2                         [115, 3]                  --
│    └─Linear: 2-3                       [115, 1]                  4
│    └─Sigmoid: 2-4                      [115, 1]                  --
Total params: 25
Trainable params: 25
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [135]:
optimizer_mlp_lim = optim.Adam(mlp_model_lim.parameters())

In [136]:
train_losses_mlp_lim = []
#test_losses  = []
train_accs_mlp_lim = []
test_accs_mlp_lim  = []

for epoch in range(1000):
    for features, labels in data_loader_lim:
        train_preds_mlp_lim = mlp_model_lim(features)
        class_weights_batch = class_weights_original[labels.long()]
        loss_function = nn.BCELoss(weight=class_weights_batch)

        train_loss_mlp_lim  = loss_function(train_preds_mlp_lim, labels)

        with torch.no_grad():
            test_preds_mlp_lim = mlp_model_lim(X_validation_tensor_lim)
            #test_loss  = loss_function(test_preds, y_test)

        train_acc = calculate_accuracy(train_preds_mlp_lim, labels)
        test_acc  = calculate_accuracy(test_preds_mlp_lim, y_validation_tensor)

        optimizer_mlp_lim.zero_grad()
        train_loss_mlp_lim.backward()

        optimizer_mlp_lim.step()

        train_losses_mlp_lim.append(train_loss_mlp_lim.item())
        #test_losses.append(test_loss.item())
        train_accs_mlp_lim.append(train_acc.item())
        test_accs_mlp_lim.append(test_acc.item())

    #if epoch%100==0:
    print(f'Epoch: {epoch} \t|' \
          f' Train loss: {np.round(train_loss_mlp_lim.item(),3)} \t|' \
              #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
          f' Train acc: {np.round(train_acc.item(),2)} \t|' \
          f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 1 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 2 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 3 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 4 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 5 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 6 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 7 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 8 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 9 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 10 	| Train loss: 0.253 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 11 	| Train loss: 0.252 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 12 	| Train loss: 0.252 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 13 	| Train loss: 0.252 	| Train acc: 0.23 	| Test acc: 0.23
Epoch: 14 	| Train loss: 0.252 	| Train acc: 0.23 	| Test 

In [137]:
evaluate_nn(y_validation_tensor.clone().detach(), test_preds_mlp_lim.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 58.67%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.799896    0.282949  0.586736     0.541422      0.678793
recall        0.613873    0.498034  0.586736     0.555954      0.586736
f1-score      0.694646    0.360874  0.586736     0.527760      0.616455
support    2494.000000  763.000000  0.586736  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[1531  963]
 [ 383  380]]


In [138]:
test_preds_mlp_lim = mlp_model_lim(X_test_tensor_lim)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_mlp_lim.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 56.95%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.768330    0.279018  0.569528     0.523674      0.645558
recall        0.608959    0.451807  0.569528     0.530383      0.569528
f1-score      0.679424    0.344986  0.569528     0.512205      0.595511
support    2478.000000  830.000000  0.569528  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1509  969]
 [ 455  375]]


In [179]:
mlpsl_model_lim = MLP(num_features=6)
summary(mlpsl_model_lim, input_size=X_train_lim.shape)

Layer (type:depth-idx)                   Output Shape              Param #
MLP                                      [115, 1]                  --
├─Sequential: 1-1                        [115, 1]                  --
│    └─Linear: 2-1                       [115, 3]                  21
│    └─ReLU: 2-2                         [115, 3]                  --
│    └─Linear: 2-3                       [115, 1]                  4
│    └─Sigmoid: 2-4                      [115, 1]                  --
Total params: 25
Trainable params: 25
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [180]:
optimizer_mlpsl_lim = optim.Adam(mlpsl_model_lim.parameters())

In [181]:
train_losses_mlpsl_lim = []
#test_losses  = []
train_accs_mlpsl_lim = []
test_accs_mlpsl_lim  = []

for epoch in range(1000):
    for features, labels in data_loader_lim:
        train_preds_mlpsl_lim = mlpsl_model_lim(features)
        class_weights_batch = class_weights_original[labels.long()]
        loss_function = nn.BCELoss(weight=class_weights_batch)

        credit_score_mlpsl_lim = features[:, credit_score_index_lim].reshape(-1, 1)
        short_term_mlpsl_lim = features[:, short_term_index_lim].reshape(-1, 1)
        annual_income_mlpsl_lim = features[:, annual_income_index_lim].reshape(-1, 1)

        rule_mlpsl_lim = torch.logical_or(torch.logical_and(credit_score_mlpsl_lim<=684.5,  annual_income_mlpsl_lim<=2489636.5), torch.logical_and(credit_score_mlpsl_lim>684.5, annual_income_mlpsl_lim<=1271242.5))
        #rule_mlpsl_lim= torch.logical_or(credit_score_mlpsl_lim>3300.5, torch.logical_and(credit_score_mlpsl_lim<=3300.5,torch.logical_and(short_term_mlpsl_lim<=0.5, annual_income_mlpsl_lim<=1405107))).float()
        
        train_loss_mlpsl_lim = semantic_loss(train_preds_mlpsl_lim, labels, rule_mlpsl_lim, class_weights_batch, 0.01)

        with torch.no_grad():
            test_preds_mlpsl_lim = mlpsl_model_lim(X_validation_tensor_lim)
            #test_loss  = loss_function(test_preds, y_test)

        train_acc = calculate_accuracy(train_preds_mlpsl_lim, labels)
        test_acc  = calculate_accuracy(test_preds_mlpsl_lim, y_validation_tensor)

        optimizer_mlpsl_lim.zero_grad()
        train_loss_mlpsl_lim.backward()

        optimizer_mlpsl_lim.step()

        train_losses_mlpsl_lim.append(train_loss_mlpsl_lim.item())
        #test_losses.append(test_loss.item())
        train_accs_mlpsl_lim.append(train_acc.item())
        test_accs_mlpsl_lim.append(test_acc.item())

    #if epoch%100==0:
    print(f'Epoch: {epoch} \t|' \
          f' Train loss: {np.round(train_loss_mlpsl_lim.item(),3)} \t|' \
              #f' Test loss: {np.round(test_loss.item(),3)} \t|' \
          f' Train acc: {np.round(train_acc.item(),2)} \t|' \
          f' Test acc: {np.round(test_acc.item(),2)}')

Epoch: 0 	| Train loss: 0.227 	| Train acc: 0.23 	| Test acc: 0.28
Epoch: 1 	| Train loss: 0.227 	| Train acc: 0.23 	| Test acc: 0.28
Epoch: 2 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 3 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 4 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 5 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 6 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 7 	| Train loss: 0.226 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 8 	| Train loss: 0.226 	| Train acc: 0.23 	| Test acc: 0.28
Epoch: 9 	| Train loss: 0.225 	| Train acc: 0.23 	| Test acc: 0.28
Epoch: 10 	| Train loss: 0.225 	| Train acc: 0.23 	| Test acc: 0.28
Epoch: 11 	| Train loss: 0.225 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 12 	| Train loss: 0.225 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 13 	| Train loss: 0.225 	| Train acc: 0.24 	| Test acc: 0.28
Epoch: 14 	| Train loss: 0.225 	| Train acc: 0.24 	| Test 

In [182]:
evaluate_nn(y_validation_tensor.clone().detach(), test_preds_mlpsl_lim.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 52.23%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.787730    0.256300   0.52226     0.522015      0.663235
recall        0.514836    0.546527   0.52226     0.530681      0.522260
f1-score      0.622696    0.348954   0.52226     0.485825      0.558568
support    2494.000000  763.000000   0.52226  3257.000000   3257.000000
_______________________________________________
Confusion Matrix: 
 [[1284 1210]
 [ 346  417]]


In [183]:
test_preds_mlpsl_lim = mlpsl_model_lim(X_test_tensor_lim)
evaluate_nn(y_test_tensor.clone().detach(), test_preds_mlpsl_lim.clone().detach().round(), train=False)

Test Result:
Accuracy Score: 51.96%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.769557    0.271248  0.519649     0.520403      0.644528
recall        0.512107    0.542169  0.519649     0.527138      0.519649
f1-score      0.614975    0.361591  0.519649     0.488283      0.551399
support    2478.000000  830.000000  0.519649  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1269 1209]
 [ 380  450]]


In [163]:
credit_score_test = torch.tensor(X_test['Credit Score'].values, dtype=torch.float32).reshape(len(X_test), 1)
short_term_test = torch.tensor(X_test['Term_Short Term'].values, dtype=torch.float32).reshape(len(X_test), 1)
annual_income_test = torch.tensor(X_test['Annual Income'].values, dtype=torch.float32).reshape(len(X_test), 1)

rule_test = torch.logical_or(
    torch.logical_and(credit_score_test <= 684.5, annual_income_test <= 2489636.5),
    torch.logical_and(credit_score_test > 684.5, annual_income_test <= 1271242.5)
)
rule_matched = torch.nonzero(rule_test.float(), as_tuple=True)[0].tolist()
yespred = torch.nonzero(test_preds_mlpsl_lim.round(), as_tuple=True)[0].tolist()
same = [value for value in yespred if value in rule_matched]
len(same)

1612

In [336]:
dt_model_lim = DecisionTreeClassifier(class_weight='balanced')
dt_model_lim.fit(X_train_lim_scaled, y_train_lim)

predictions_dt_lim = dt_model_lim.predict(X_test_lim_scaled)

evaluate_nn(y_test_tensor.clone().detach(), predictions_dt_lim.round(), train=False)

Test Result:
Accuracy Score: 61.76%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.751973    0.258602  0.617594     0.505287      0.628183
recall        0.730428    0.280723  0.617594     0.505575      0.617594
f1-score      0.741044    0.269209  0.617594     0.505126      0.622657
support    2478.000000  830.000000  0.617594  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1810  668]
 [ 597  233]]


In [229]:
xgb_model_lim = xgb.XGBClassifier(scale_pos_weight=(len(y_train_lim) - sum(y_train_lim)) / sum(y_train_lim))

xgb_model_lim.fit(X_train_lim_scaled, y_train_lim)

predictions_xgb_lim = xgb_model_lim.predict(X_test_lim_scaled)

evaluate_nn(y_test_tensor.clone().detach(), predictions_xgb_lim.round(), train=False)


Test Result:
Accuracy Score: 63.81%
_______________________________________________
CLASSIFICATION REPORT:
                   0.0         1.0  accuracy    macro avg  weighted avg
precision     0.763255    0.290286   0.63815     0.526770      0.644584
recall        0.749395    0.306024   0.63815     0.527709      0.638150
f1-score      0.756261    0.297947   0.63815     0.527104      0.641267
support    2478.000000  830.000000   0.63815  3308.000000   3308.000000
_______________________________________________
Confusion Matrix: 
 [[1857  621]
 [ 576  254]]


# Creating conformal predictions

In [254]:
n = len(X_calibration)
predictions = lr_model(X_calibration_tensor)
true_class_probs = torch.where(y_calibration_tensor == 1, predictions, 1 - predictions)
scores = 1-true_class_probs

In [267]:
alpha = 0.05

q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level, method='higher')
qhat

0.7777948

In [261]:
plt.hist(predictions.detach().numpy(), bins=30, range=(0, 1))
plt.xlabel("1 - s(y,x)")
plt.ylabel("Frequency")
plt.title("Histogram of non-conformity scores")
plt.show()


AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [268]:
prediction_sets = []

for prob in predictions_xgb_lim_proba[:,1]:
    prediction_set = []
    if prob > 1-qhat:
        prediction_set.append("Charged Off")
    if 1 - prob > 1-qhat:
        prediction_set.append("Fully Paid")
    prediction_sets.append(prediction_set)

print(prediction_sets)

[['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Fully Paid'], ['Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Fully Paid'], ['Charged Off', 'Ful

In [269]:
count_length_1 = 0
count_length_2 = 0

for prediction_set in prediction_sets:
    if len(prediction_set) == 1:
        count_length_1 += 1
    elif len(prediction_set) == 2:
        count_length_2 += 1

print("Number of prediction sets of length 1:", count_length_1)
print("Number of prediction sets of length 2:", count_length_2)


Number of prediction sets of length 1: 742
Number of prediction sets of length 2: 2566


In [266]:
predictions = xgb_model.predict_proba(X_calibration_scaled)
predictions_xgb_lim_proba = xgb_model.predict_proba(X_test_scaled)
y_calibration = np.array(y_calibration)

true_class_probs = np.array([predictions[i, y_calibration[i]] for i in range(len(y_calibration))])

scores = 1 - true_class_probs

print("Predictions shape:", predictions.shape)
print("True class probabilities shape:", true_class_probs.shape)
print("Uncertainty scores shape:", scores.shape)

print("Sample predictions:", predictions[:5])
print("Sample true class probabilities:", true_class_probs[:5])
print("Sample uncertainty scores:", scores[:5])


Predictions shape: (3307, 2)
True class probabilities shape: (3307,)
Uncertainty scores shape: (3307,)
Sample predictions: [[0.742601   0.25739902]
 [0.50895983 0.49104017]
 [0.66303027 0.33696973]
 [0.61551404 0.38448593]
 [0.43267548 0.5673245 ]]
Sample true class probabilities: [0.742601   0.50895983 0.66303027 0.38448593 0.43267548]
Sample uncertainty scores: [0.25739902 0.49104017 0.33696973 0.61551404 0.5673245 ]


In [270]:
def calculate_coverage(prediction_sets, y_test):
    correct_predictions = 0

    for i, prediction_set in enumerate(prediction_sets):
        if y_test[i] in prediction_set:
            correct_predictions += 1

    coverage_percentage = (correct_predictions / len(y_test)) * 100
    return coverage_percentage

In [278]:

predictions = xgb_model.predict_proba(X_calibration_scaled)
predictions_xgb_proba = xgb_model.predict_proba(X_test_scaled)
y_calibration = np.array(y_calibration)

true_class_probs = np.array([predictions[i, y_calibration[i]] for i in range(len(y_calibration))])

scores = 1 - true_class_probs

alpha = 0.01

q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level, method='higher')

prediction_sets=[]

for prob in predictions_xgb_lim_proba[:,1]:
    prediction_set = []
    if prob > 1-qhat:
        prediction_set.append(1)
    if 1 - prob > 1-qhat:
        prediction_set.append(0)
    prediction_sets.append(prediction_set)

coverage = calculate_coverage(prediction_sets, y_test.tolist())
print(f"Coverage: {coverage:.2f}%")


Coverage: 99.09%


In [300]:
probabilities = test_preds_mlpsl_lim
threshold = 0.95

high_confidence_mask = (probabilities >= threshold) | (probabilities <= 1 - threshold)

binary_predictions = (probabilities >= 0.5).float()

correct_high_confidence_predictions = (binary_predictions[high_confidence_mask] == y_test_tensor[high_confidence_mask]).float()
coverage = correct_high_confidence_predictions.mean().item()
total_high_confidence = high_confidence_mask.sum().item()

print(f"Coverage: {round(coverage, 4)}")
print(f"Number of high-confidence predictions: {total_high_confidence}")

Coverage: 0.7576
Number of high-confidence predictions: 66


In [316]:
n = len(X_calibration)
predictions = mlpsl_model_lim(X_calibration_tensor_lim)
true_class_probs = torch.where(y_calibration_tensor == 1, predictions, 1 - predictions)
scores = 1-true_class_probs

alpha = 0.05

q_level = np.ceil((n+1)*(1-alpha))/n
#qhat = np.quantile(scores.clone().detach(), q_level, method='higher')
qhat = np.quantile(scores.detach().clone().numpy(), q_level, method='higher')

prediction_sets = []

for prob in test_preds_mlpsl_lim:
    prediction_set = []
    if prob > 1-qhat:
        prediction_set.append(1)
    if 1 - prob > 1-qhat:
        prediction_set.append(0)
    prediction_sets.append(prediction_set)

coverage = calculate_coverage(prediction_sets, y_test.tolist())
print(f"Coverage: {coverage:.2f}%")

Coverage: 94.56%


In [215]:
rounded_preds_lr = torch.round(test_preds_mlp_lim)
rounded_preds_sl = torch.round(test_preds_mlpsl_lim)

different_predictions = (rounded_preds_lr != rounded_preds_sl)

correct_predictions_lr = (rounded_preds_lr == y_test_tensor)
correct_predictions_sl = (rounded_preds_sl == y_test_tensor)

different_and_correct_lr = different_predictions & correct_predictions_lr
different_and_correct_sl = different_predictions & correct_predictions_sl

indices_different_and_correct_lr = torch.nonzero(different_and_correct_lr).flatten()
indices_different_and_correct_sl = torch.nonzero(different_and_correct_sl).flatten()

only_sl_correct = torch.nonzero(different_and_correct_sl.float(), as_tuple=True)[0].tolist()
only_lr_correct = torch.nonzero(different_and_correct_lr.float(), as_tuple=True)[0].tolist()

In [217]:
only_correct = only_lr_correct
condition1 = (X_test.iloc[only_correct]['Credit Score'] <= 684.5) & (X_test.iloc[only_correct]['Annual Income'] <= 2489636.5)
condition2 = (X_test.iloc[only_correct]['Credit Score'] > 684.5) & (X_test.iloc[only_correct]['Annual Income'] <= 1271242.5)

combined_condition = condition1 | condition2

print('All the cases: ',len(X_test.iloc[only_correct][['Credit Score', 'Annual Income']]))
print('Meet the rule: ',len(X_test.iloc[only_correct][combined_condition]))
#print('Meet the rule: ',sum(X_test.iloc[only_lr_correct]['Credit Score']<652.5))
print('True class: ',sum(y_test.iloc[only_correct]))


All the cases:  370
Meet the rule:  200
True class:  9
